In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%capture --no-stderr
!pip install python-dotenv langchain_openai langchain-chroma pypdf langchain langchain_community

In [ ]:
# 환경변수 설정

In [ ]:
# 라이브러리 불러오기
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv("/content/.env")
# 환경 변수에서 API 키 가져오기
api_key = os.getenv("OPENAI_API_KEY")

# **2024 부동산 보고서 RAG 챗봇**

In [ ]:
# 라이브러리 불러오기
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_opneai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain.schema.output_parser import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory

# **PDF 문서 로드 및 텍스트 분할**

In [ ]:
loader = PyPDFLoader("pdf 경로")
# PDF 파일에서 문서 로드
documents = loader.load()
# 텍스트 분할 설정: 청크 크기와 겹칩 설정
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# 문서를 작은 청크로 분할
chunks = text_splitter.split_documents(documents)
# 분할된 청크 수 확인
print('분할된 청크의 수:', len(chunks))

# **임베딩 생성 및 Chroma 데이터베이스 저장**

In [ ]:
# 임베딩 생성 및 Chroma 데이터베이스에 저장
# OpenAI의 임베딩 모델 설정
embedding_function = OpenAIEmbeddings()
persist_directory = "크로마 DB 저장 경로"

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_function,
    # DB 저장 경로
    persist_directory=persist_directory,
)

print('문서의 수:', vectorstore._collection.count())

# **검색 및 재정렬**

In [ ]:
# 리트리버 생성
# 상위 3개의 관련 문서 검색 설정
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [ ]:
# 프롬프트 템플릿 설정: 사용자 질문에 대한 답변을 생성하기 위한 템플릿
template = """당신은 KB 부동산 보고서 전문가입니다. 다음 정보를 바탕으로 사용자의 질문에 답변해주세요.
컨텍스트: {context}
"""

# 프롬프트 템플릿 설정
prompt = ChatPromptTempalte.from_messages(
    [
        ("system", template),
        ("placeholder", "{chat_history}"),
        ("human", "{question}")
    ]
)

# AI 모델 설정
model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [ ]:
print(prompt.format(context="컨텍스트 예시"), chat_history=["대화 기록 예시1", "대화 기록 예시2"], question="질문 예시")